In [21]:
import sys
import time
from geopy.geocoders import Nominatim, GoogleV3, Bing
import folium
from bs4 import BeautifulSoup
import requests

import numpy as np
import googleCreds
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn import preprocessing
import json
import csv
from urllib.request import urlopen

def getCompaniesDataframe(googleAPIKey,
                          jobName = "data scientist",
                          locationName = "Milano Lombardia",
                          resource = "https://it.indeed.com/",
                          maxPages = 1,
                          dfPath = "data/dfCompanies.pkl",
                          savePickle = True,
                          dropNAs = True):

    """

    The function returns a dataframe with following features:

    - company name (companyName): str
    - job name (jobName): str
    - link to the description of the position (link): str
    - latitude of company's headquarters based on Google Maps Search (lt): float
    - longitude of company's headquarters based on Google Maps Search (lg): float
    - address of company's headquarters based on Google Maps Search (address): str

    If getting one of the features was unsuccesful it is replaced by np.nan (numpy NAN).

    Inputs:
        googleAPIKey(str):  path to Google Maps API key

        jobName     (str):  job name for the search,
                            default: "data scientist"

        locaionName (str):  location name,
                            default: "Milano%2C+Lombardia"

        resource    (str):  URL of job source, by now available only default value,
                            default: "https://it.indeed.com/"

        maxPages    (int):  number of pages of results to scan,
                            default: 1

        dfPath      (str):  path to save a dataframe as CSV file,
                            default: "dfCompanies.csv"

        saveCSV     (bool): True, if csv file should be saved to dfPath,
                            default: True

        dropNAs     (bool): True, if rows containing np.nan values should be dropped from a dataframe,
                            default: True
    Outputs:

        pandas DataFrame
    """

    jobName = "+".join(jobName.split())
    city = locationName.split()[0]
    locationName = "%2C+".join(locationName.split())

    #To be extended if other resources are used
    if resource == "https://it.indeed.com/":
        if maxPages>1:
            urlList = [f"https://it.indeed.com/offerte-lavoro?q={jobName}&l={locationName}"]
            urlList+=[f"https://it.indeed.com/jobs?q={jobName}&l={locationName}&start={i}0" for i in range(1,maxPages+1)]
        else:
            urlList = [f"https://it.indeed.com/offerte-lavoro?q={jobName}&l={locationName}"]


    companyInfo = {"companyName":[],
                   "jobName":[],
                   "link":[],
                   "lt" : [],
                   "lg" : [],
                   "address" : []}

    for url in urlList:
        session = requests.session()
        response = session.get(url)

        soup = BeautifulSoup(response.text, 'html.parser')

        for element in soup.find_all('span', class_='company'):
            #print(element)
            #input()
            try:
                name = element.find('a', class_="turnstileLink").text.strip()
                #print(name)
            except:
                name = element.text.strip()
                #print(name)
            companyInfo["companyName"].append(name)

        for element in soup.find_all('h2', class_='title'):
            #print(element)
            #print(element.find('a').get("href"))
            try:
                currentLink = element.find('a').get("href")
                companyInfo["link"].append("https://it.indeed.com"+currentLink)
                #print("Link: "+"https://it.indeed.com"+currentLink)
            except:
                print("Error adding link")
                companyInfo["link"].append(np.nan)
                #print(element)
                #input()
            try:
                jobName = element.find('a', class_="jobtitle turnstileLink ").text.strip()
                #print(jobName)
                #print("JobName Try Branch")
            except:
                jobName = element.text.strip()
                #print(jobName)
                #print("JobName Except Branch")
            #print(jobName)
            companyInfo["jobName"].append(jobName)

    #Sanity check if any info was lost/missed
    assert len(companyInfo["companyName"]) == len(companyInfo["jobName"]) == len(companyInfo["link"])

    # In order to get companies' locations I need their address which I can get using Google Maps API.
    # I had to create an account for that purpose.
    # Credentials are stored in googleCreds.py for security reasons.
    geolocator = GoogleV3(api_key = googleAPIKey)

    for company in companyInfo["companyName"]:
        rawName = str(company+" Milano").replace(" ","+")
        url = 'https://maps.googleapis.com/maps/api/geocode/json?'+'address={}&key={}'.format(rawName,googleCreds.GOOGLE_API_KEY)
        p = {'address' : 'Milano'}
        r = requests.get(url, params = p).json()

        time.sleep(0.5)
        try:
            results = r['results']
            results = results[0]

            location = geolocator.geocode(results["formatted_address"])
            if "Milan" in location.address:
                print(company + " was succesfully added with address:\n" + location.address + "\n")

                companyInfo["lt"].append(location.latitude)
                companyInfo["lg"].append(location.longitude)
                companyInfo["address"].append(location.address)
            else:
                print(company + " was skipped. "+location.address+" is not in Milano\n")
                companyInfo["lt"].append(np.nan)
                companyInfo["lg"].append(np.nan)
                companyInfo["address"].append(np.nan)
        except:
            print(company + " was skipped. Probably was not found\n")
            companyInfo["lt"].append(np.nan)
            companyInfo["lg"].append(np.nan)
            companyInfo["address"].append(np.nan)

    #Sanity check if any info was lost/missed
    assert len(companyInfo["companyName"]) == len(companyInfo["jobName"]) == len(companyInfo["link"])
    assert len(companyInfo["lt"]) == len(companyInfo["lg"]) == len(companyInfo["address"]) == len(companyInfo["link"])
    print("Assertion succesful. Creating Dataframe")

    dfCompanies = pd.DataFrame(companyInfo)
    dfCompanies = dfCompanies[dfCompanies["address"].astype(str).str.contains(city)]

    if dropNAs:
        dfCompanies.dropna(inplace = True)

    if savePickle:
        dfCompanies.to_pickle(dfPath)

    return dfCompanies


# Green Zones
# The next step is to analyze how green each zone of Milan is. I used data on green zones in order to obtain Vegetation Concentration (area of green zones relative to total area).
def getGreenZonesDataframe(resource = "https://dati.comune.milano.it/dataset/da6bc86d-c27f-4256-84eb-86c35dad7d0e/resource/bd55cd57-4dd7-4d72-b182-de4f4b7de8c6/download/ds339_territorioambiente_aree-verdi-zona-superficie_2014.csv",
                          dfPath = "data/",
                          savePickle = True):

    """

    The function returns a dataframe with following features:
    - City zone number (ZONADEC): int
    - Total area of green zones (GreenArea): float
    - Total area of the city zone (TotalArea): float
    - Concentration of green calculated as GreenArea/TotalArea (GreenConc): float

    Source: https://dati.comune.milano.it/dataset/ds339-territorioambiente-aree-verdi-zona-superficie-2014
    Last Access Date: 11/08/2020

    Inputs:

        resource    (str):  URL of source, by now only default value should be used,
                            default: "https://dati.comune.milano.it/dataset/da6bc86d-c27f-4256-84eb-86c35dad7d0e/resource/bd55cd57-4dd7-4d72-b182-de4f4b7de8c6/download/ds339_territorioambiente_aree-verdi-zona-superficie_2014.csv"

        dfPath      (str):  path to save dataframes as pickle files,
                            default: "data/"

        savePickle  (bool): True, if pickle files should be saved to dfPath,
                            default: True


    Outputs:

        pandas DataFrame
    """

    dfGreenRaw = pd.read_csv(resource,
                         header = 0, sep = ";", encoding='latin-1')


    dfGreenRaw["Superficie totale in mq"] = dfGreenRaw["Superficie totale in mq"].apply(lambda x: float(str(x).replace(",",".")))




    dfGreenRaw = dfGreenRaw.groupby(by="Zona").sum()
    dfGreenRaw = dfGreenRaw.reset_index()




    # Getting info on total area of each zone from Wikipedia

    # In[18]:


    dfZones = pd.read_html('https://en.wikipedia.org/wiki/Municipalities_of_Milan',
                              flavor='bs4')
    dfZones = dfZones[1][:9]



    # Merging two Data frames and calculating the new column

    dfGreen = pd.concat([dfGreenRaw, dfZones],axis = 1)[["Zona","Superficie totale in mq","Area(km2)"]]
    #Had to rename column because geojson file had ZONADEC variable for zone number.
    #I'll need that when creating a map.
    dfGreen.columns = ["ZONADEC","GreenArea", "TotalArea"]

    #Converting m2 to km2
    dfGreen["GreenArea"] = dfGreen["GreenArea"]/1000000

    dfGreen["GreenConc"] = dfGreen["GreenArea"]/dfGreen["TotalArea"]

    if savePickle:
        dfGreenRaw.to_pickle(dfPath+'dfGreenRaw.pkl')
        dfZones.to_pickle(dfPath+'dfZones.pkl')
        dfGreen.to_pickle(dfPath+"dfGreen.pkl")

    return dfGreen

# ### 5. Air Quality
# In this section I analyze data on contamination for 2019 and 2020 obtained from stations located in Milano.
# I will calculate an Eco score which is an inverted value of mean normalized concentrations of contaminants.

# In[38]:


def getAirQualityDataframe(resource = ['https://dati.comune.milano.it/dataset/3e752fec-06fd-421b-ae9b-4d5d7a177640/resource/698a58e6-f276-44e1-92b1-3d2b81a4ad47/download/qaria_datoariagiornostazione_2020-01-08.csv',
                                      'https://dati.comune.milano.it/dataset/ccf8b61d-728f-46e7-bee9-e685c7b6cd35/resource/88c1e729-420e-433f-9397-875b54aa471d/download/qaria_datoariagiornostazione_2021-01-01.csv'],
                          dfPath = "data/",
                          savePickle = True,
                          plot = False):

    """
    The function returns a dataframe with following features:
    - Station number (id_amat): int
    - Absolute median value of contaminant concentrations (valore): float
    - Median value of scaled contaminant concentrations (normv): float
    - Station name (name): str
    - Coordinates of a station as a list [latitude, longitude] (coords): list of floats
    - Latitude of a station (lt): float
    - Longitude of a station (lg): float


    Inputs:

        resource    (list of str):  URL of source, by now only default value should be used,
                            default: "https://dati.comune.milano.it/dataset/da6bc86d-c27f-4256-84eb-86c35dad7d0e/resource/bd55cd57-4dd7-4d72-b182-de4f4b7de8c6/download/ds339_territorioambiente_aree-verdi-zona-superficie_2014.csv"

        dfPath      (str):  path to save dataframes as pickle files,
                            default: "data/"

        savePickle  (bool): True, if pickle files should be saved to dfPath,
                            default: True

        plot  (bool):       if True, html bar plots for each station will be saved to dfPath,
                            default: False

    Outputs:

        pandas DataFrame
    """

    for i,r in enumerate(resource):
        if i==0:
            dfAirRaw = pd.read_csv(r, sep=None)
        else:
            dfT = pd.read_csv(r, sep=None)
            dfAirRaw = pd.concat([dfAirRaw,dfT])

    dfAirRaw.dropna(inplace = True)

    dfAir = dfAirRaw
    dfAir['data']=pd.to_datetime(dfAir['data'], format='%Y-%m-%d')
    dfAir["normv"] = dfAir['valore'] / dfAir.groupby('inquinante')['valore'].transform('max')

    dfAir = dfAir.groupby(by="stazione_id").median()

    dfAir = dfAir.reset_index()
    dfAir.columns = ["id_amat", "valore", "normv"]


    # Retrieving data from geojson for stations.
    #Source: https://dati.comune.milano.it/dataset/ds484_stazioni_di_monitoraggio_inquinanti_atmosferici_dellarpa_sit/resource/635c6508-b335-48b1-b3c8-d43e78ad3380

    stationsPath = 'https://dati.comune.milano.it/dataset/d6960c75-0a02-4fda-a85f-3b1c4aa725d6/resource/635c6508-b335-48b1-b3c8-d43e78ad3380/download/qaria_stazione.geojson'

    # with open(stationsPath) as data_file:
    #     data = json.load(data_file)
    stationsData = urlopen(stationsPath)
    stationsData = json.load(stationsData)
    dctStations = {"id_amat":[],
                   "name" : [],
                   "coords" : []}

    for i in stationsData["features"]:
        dctStations["id_amat"].append(i["properties"]["id_amat"])
        dctStations["name"].append(i["properties"]["nome"])
        dctStations["coords"].append([i["geometry"]["coordinates"][1], i["geometry"]["coordinates"][0]])

    dfStations = pd.DataFrame(dctStations)
    dfAirStations = dfAir.merge(dfStations, left_on='id_amat', right_on='id_amat')



    # Need to separate the coordinates for further modelling
    dfAirStations["lt"] = dfAirStations["coords"].map(lambda x: x[0])
    dfAirStations["lg"] = dfAirStations["coords"].map(lambda x: x[1])



    if savePickle:
        dfAirRaw.to_pickle(dfPath+'dfAirRaw.pkl')
        dfAir.to_pickle(dfPath+'dfAir.pkl')
        dfStations.to_pickle(dfPath+"dfStations.pkl")
        dfAirStations.to_pickle(dfPath+"dfAirStations.pkl")

    if plot:
        for station in dfAirRaw["stazione_id"].unique():
            f = px.bar(dfAirRaw, x='data', y='normv', color='inquinante', title=f"Station #{station}")
            f.write_html(dfPath+f"Air_Station_{station}.html")
    return dfAirStations


def getLoc(address, annot = False, api_key = googleCreds.GOOGLE_API_KEY):
    """
    The function returns coordinates of a given address.

    Input:
    address (str): address
    annot (bool): if True, messages on result are displayed
    api_key (str): API key for GoogleV3 geocoder,
                   can be obtained at https://console.cloud.google.com/ (look for Geocoding API)

    Output:
    list(float): [latitude, longitude] or np.nan if not found
    """
    try:
        geolocator = GoogleV3(api_key = api_key)
        location = geolocator.geocode(address)
        if annot:
            print(address + " was succesfully added\n")
        return [location[1][0], location[1][1]]

    except:
        if annot:
            print(address + " was skipped. Probably was not found\n")
        return np.nan

def getAccommodationDF(minPrice = 500, maxPrice = 5000, maxPages = 10, dfPath = "data/", savePickle = True):

    """
    The function returns a dataframe with following features:
    - Accommodation id (id): str
    - Accommodation type (type): str
    - Accommodation address (address): str
    - Accommodation price (price): float
    - Coordinates of an accommodation as a list [latitude, longitude] (coords): list of floats

    The data is scrapped from: https://www.immobiliare.it/

    Inputs:

    minPrice    (float): minimum price (rent),
                default: 500

    maxPrice    (float): maxmum price (rent),
                default: 5000

    maxPages    (float): number of result pages to be processed (50 items per page),
                default: 10

    dfPath      (str):  path to save dataframes as pickle files,
                default: "data/"

    savePickle  (bool): True, if pickle files should be saved to dfPath,
          default: True

    Outputs:

    pandas DataFrame
    """

    urlList = [f"https://www.immobiliare.it/affitto-case/milano/?criterio=rilevanza&prezzoMinimo={minPrice}&prezzoMassimo={maxPrice}"]
    urlList+=[f"https://www.immobiliare.it/affitto-case/milano/?criterio=rilevanza&prezzoMinimo={minPrice}&prezzoMassimo={maxPrice}&pag={i+1}" for i in range(maxPages)]

    objects = {"id":[],
               "type":[],
               "address":[],
               "price":[]}

    for url in urlList:
        session = requests.session()
        response = session.get(url+"&boxAuto[]=1&boxAuto[]=4")

        soup = BeautifulSoup(response.text, 'html.parser')

        for element in soup.find_all('p', class_='titolo text-primary'):
            try:
                name = element.text.strip().split()
                objects["id"].append(element.find('a').get("id"))
                objects["type"].append(name[0])
                objects["address"].append(" ".join(name[1:]))
            except:
                pass

        for element in soup.find_all('li', class_='lif__item lif__pricing'):
            try:
                if element.find('div')==None:

                    price = element.text.strip().split()[1]
                    price= price.replace(".","")
                    objects["price"].append(int(price))
                else:

                    price = element.find('div').text.strip().split()[1]
                    price= price.replace(".","")

                    objects["price"].append(int(price))
            except:
                pass

            #Sanity check
    assert len(objects["id"]) == len(objects["type"]) == len(objects["address"]) == len(objects["price"])
    print(f"All successful. {len(objects['id'])} objects has been added")

    dfObjects = pd.DataFrame(objects)
    dfObjects["coords"] = dfObjects["address"].map(getLoc)
    dfObjects.dropna(inplace = True)
    if savePickle:
        dfObjects.to_pickle(dfPath+'dfObjects.pkl')
    return dfObjects


#
# # ### Defining Zone for each object
# import json
# from shapely.geometry import shape, Point
# with open('data/zonedecentramento.geojson') as f:
#     js = json.load(f)
#
#
# dfObjects["Zone"] = dfObjects["coords"].map(lambda x: getZone(x, js))
#
#
# dfObjects= pd.merge(dfObjects, dfGreen, left_on='Zone', right_on="ZONADEC", right_index=False, how='left', sort=False)
#
#
#
# dfObjects.drop(columns=["TotalArea","GreenArea", "ZONADEC"], inplace = True)
#
# dfObjects.columns = list(dfObjects.columns[:-1])+["GreenConc"]
#
# dfObjects["lt"] = dfObjects["coords"].map(lambda x: x[0])
# dfObjects["lg"] = dfObjects["coords"].map(lambda x: x[1])
#
#
# dfObjects.to_csv("dfObjectsUnscored.csv", index = False)
#
#
# airWeight, safetyWeight, jobWeight, greenWeight, priceWeight  = 5, 5, 5, 5, 5
#
# dfObjects[["price"]].hist()
#
#
# # Let's scale it and transform using log function.
# scaler = preprocessing.MinMaxScaler()
# dfObjects["priceLogScaled"] = scaler.fit_transform(np.log(dfObjects[["price"]]))
# numColumns = ["priceLogScaled", "contamination", "distanceToDangerZone",
#                      "distanceToMedianJobLocation","GreenConc"]
# dfObjects["totalScore"]=dfObjects.apply(lambda x: totalScore([x[i] for i in numColumns]), axis=1)
# dfObjects
#
#
# # In[ ]:
#
#
# dfObjects["priceLogScaled"].hist()
#
#
# # ### Saving DF
# dfObjects.to_csv("dfObjects.csv", index = False)
#
#
# # ### Creating a model for predicting price
#
# # #### Leaving only numerical columns
#
# # In[ ]:
#
#
# numColumns = ["price", "contamination", "distanceToDangerZone",
#                      "distanceToMedianJobLocation","GreenConc"]
# dfFinal = dfObjects[["price", "contamination", "distanceToDangerZone",
#                      "distanceToMedianJobLocation","GreenConc"]]
# dfFinal
#
#
# # #### Scaling
#
# # In[ ]:
#
#
# from sklearn import preprocessing
# scaler = preprocessing.MinMaxScaler()
# scaled = scaler.fit_transform(dfFinal)
# dfFinal = pd.DataFrame(scaled)
# dfFinal.columns = numColumns
#
#
#
#
# priceWeight = 5
# dfFinal["totalScore"]=dfFinal.apply(lambda x: totalScore([x[i] for i in dfFinal.columns]), axis=1)
# dfFinal
#
#
#
#
# dfObjectsScored = pd.concat([dfObjects[["id", "type", "address", "price","coords","lt","lg"]], dfFinal[["totalScore"]]], axis = 1)
#
#
#
#
# dfObjectsScored.to_csv("dfObjectsScored.csv", index = False)


In [3]:
pip install selenium

     |████████████████████████████████| 904 kB 4.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
ls /home/inotin/Documents/

geckodriver*


In [24]:
pip install careerjet-api-client

  Created wheel for careerjet-api-client: filename=careerjet_api_client-3.0.1-py3-none-any.whl size=4254 sha256=1876476d4bc48c6e903ba7f2667c90452032e2de28f9257680c78dd6829330da
  Stored in directory: /home/inotin/.cache/pip/wheels/50/48/cb/a4ce98e9b5be16fbcfe49465cf438cd87aae054cd6ad17db32
Successfully built careerjet-api-client
Note: you may need to restart the kernel to use updated packages.


In [16]:

from selenium import webdriver
driver = webdriver.Firefox()


In [20]:
import re

In [23]:
companyInfo = {"companyName":[],
               "jobName":[],
               "link":[],
               "lt" : [],
               "lg" : [],
               "address" : []}
for url in urlList:
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

   
    for element in soup.find_all('a', id=re.compile('^job_')):
    #     print(element)

        print(element['href'])
        jobName = element.find('span', class_=lambda x: x != 'label').text
        print(jobName)
        companyInfo["link"].append("https://it.indeed.com"+str(element['href']))
        companyInfo["jobName"].append(jobName)
        name = element.find('span', class_='companyName').text
        print(name)
        companyInfo["companyName"].append(name)
        print('='*10)

/rc/clk?jk=116332180c9ab20f&fccid=d47acb5b5596d9e4&vjs=3
Agent-Guest Services
Marriott International, Inc
/rc/clk?jk=6d9922699fbf799f&fccid=ac6d93e2692054b1&vjs=3
JUNIOR INTERIOR DESIGNER
Global Planning Architecture
/rc/clk?jk=244d1c3cf1225823&fccid=a8c536c3d65bf125&vjs=3
Architetto Designer / Luxury Brand
Neon Ferrari Trada
/rc/clk?jk=0074a17579ca296e&fccid=559dbeaee2b65abb&vjs=3
Graphic designer
Vectorealism
/company/Intech-Srl---INTERFACEGLOBE/jobs/Web-Graphic-Designer-Freelance-369095fdd0ed1cc7?fccid=be7258780ffdce01&vjs=3
Web and Graphic Designer Freelance
Intech Srl - INTERFACEGLOBE
/rc/clk?jk=ae7a875ef1dbe562&fccid=8e0ce4d27f76a735&vjs=3
Graphic designer [Freelance]
Figmenta Srl.
/company/IVH-Group-Spa/jobs/Stage-Graphic-Designer-Linguaggio-HTML-5d519d81ea835ef3?fccid=6c56210295d6992a&vjs=3
Stage Graphic designer e Linguaggio HTML
IVH Group Spa
/rc/clk?jk=66b87d14319453f3&fccid=b71ee43710135ed5&vjs=3
PRODUCT DESIGNER
PREMI S.P.A.
/rc/clk?jk=5247edfc7369f5aa&fccid=0227baf05c4296

In [18]:
jobName = 'designer'
jobName = "+".join(jobName.split())
locationName= 'Milano Lombardia'
city = locationName.split()[0]
locationName = "%2C+".join(locationName.split())
resource = "https://it.indeed.com/"
maxPages=2
#To be extended if other resources are used
if resource == "https://it.indeed.com/":
    if maxPages>1:
        urlList = [f"https://it.indeed.com/offerte-lavoro?q={jobName}&l={locationName}"]
        urlList+=[f"https://it.indeed.com/jobs?q={jobName}&l={locationName}&start={i}0" for i in range(1,maxPages+1)]
    else:
        urlList = [f"https://it.indeed.com/offerte-lavoro?q={jobName}&l={locationName}"]


print(urlList)

['https://it.indeed.com/offerte-lavoro?q=designer&l=Milano%2C+Lombardia', 'https://it.indeed.com/jobs?q=designer&l=Milano%2C+Lombardia&start=10', 'https://it.indeed.com/jobs?q=designer&l=Milano%2C+Lombardia&start=20']


In [5]:
import re
companyInfo = {"companyName":[],
               "jobName":[],
               "link":[],
               "lt" : [],
               "lg" : [],
               "address" : []}
# input(urlList)
for url in urlList:
    session = requests.session()
    response = session.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
   
    for element in soup.find_all('a', id=re.compile('^job_')):
    #     print(element)

        print(element['href'])
        jobName = element.find('span', class_=lambda x: x != 'label').text
        print(jobName)
        companyInfo["link"].append("https://it.indeed.com"+str(element['href']))
        companyInfo["jobName"].append(jobName)
        name = element.find('span', class_='companyName').text
        print(name)
        companyInfo["companyName"].append(name)
        print('='*10)

/rc/clk?jk=0074a17579ca296e&fccid=559dbeaee2b65abb&vjs=3
Graphic designer
Vectorealism
/rc/clk?jk=116332180c9ab20f&fccid=d47acb5b5596d9e4&vjs=3
Agent-Guest Services
Marriott International, Inc
/rc/clk?jk=6d9922699fbf799f&fccid=ac6d93e2692054b1&vjs=3
JUNIOR INTERIOR DESIGNER
Global Planning Architecture
/rc/clk?jk=3b099edad70681da&fccid=dd616958bd9ddc12&vjs=3
Product Designer (MI)
azienda/studio: Dainelli Studio
/rc/clk?jk=244d1c3cf1225823&fccid=a8c536c3d65bf125&vjs=3
Architetto Designer / Luxury Brand
Neon Ferrari Trada
/rc/clk?jk=f29edd3cf725d468&fccid=866cab71fab682c0&vjs=3
Graphic Designer
Balich Worldwide Shows
/rc/clk?jk=4d72086ffafd4f35&fccid=6dada33a272e0615&vjs=3
PRODUCT DESIGNER
Premi Spa
/rc/clk?jk=d1eea94395b8fc7e&fccid=a49d99f2875604a1&vjs=3
Digital Graphic Designer
Luxottica Group
/rc/clk?jk=5247edfc7369f5aa&fccid=0227baf05c429666&vjs=3
Junior Graphic Designer
Iliad Italia S.p.A.
/rc/clk?jk=e2d8108a741327c7&fccid=ca2b244e2a15ca5c&vjs=3
Motion Grapher Designer
Experis
/rc/c

In [7]:
print(len(companyInfo["companyName"]))
print(len(companyInfo["jobName"]))
print(len(companyInfo["link"]))
# input()
assert len(companyInfo["companyName"]) == len(companyInfo["jobName"]) == len(companyInfo["link"])

# In order to get companies' locations I need their address which I can get using Google Maps API.
# I had to create an account for that purpose.
# Credentials are stored in googleCreds.py for security reasons.
geolocator = GoogleV3(api_key = googleCreds.GOOGLE_API_KEY)

for company in companyInfo["companyName"]:
    print(company)
    rawName = str(company+" Milano").replace(" ","+")

    url = 'https://maps.googleapis.com/maps/api/geocode/json?'+'address={}&key={}'.format(rawName,googleCreds.GOOGLE_API_KEY)
    p = {'address' : 'Milano'}
    reqStatus = None
    attemptsLimit = 10
    numberOfAttempt = 0
    # while reqStatus!='OK' or numberOfAttempt<=attemptsLimit:
    r = requests.get(url, params = p).json()
    reqStatus=r['status']
    numberOfAttempt+=1
    print(reqStatus)
    input()
    time.sleep(0.5)
    #print(f'Attempt {numberOfAttempt} for {company}')
    # try:

    results = r['results']
    try:
        results = results[0]

        location = geolocator.geocode(results["formatted_address"])
        if "Milan" in location.address:
            print(company + " was succesfully added with address:\n" + location.address + "\n")

            companyInfo["lt"].append(location.latitude)
            companyInfo["lg"].append(location.longitude)
            companyInfo["address"].append(location.address)
        else:
            print(company + " was skipped. "+location.address+" is not in Milano\n")
            companyInfo["lt"].append(np.nan)
            companyInfo["lg"].append(np.nan)
            companyInfo["address"].append(np.nan)
    except:
        print(company + " was skipped. Probably was not found\n")
        companyInfo["lt"].append(np.nan)
        companyInfo["lg"].append(np.nan)
        companyInfo["address"].append(np.nan)

#Sanity check if any info was lost/missed
assert len(companyInfo["companyName"]) == len(companyInfo["jobName"]) == len(companyInfo["link"])
assert len(companyInfo["lt"]) == len(companyInfo["lg"]) == len(companyInfo["address"]) == len(companyInfo["link"])
print("Assertion succesful. Creating Dataframe")

dfCompanies = pd.DataFrame(companyInfo)
dfCompanies = dfCompanies[dfCompanies["address"].astype(str).str.contains(city)]


dfCompanies.dropna(inplace = True)
dfCompanies

45
45
45
Vectorealism
OK

Vectorealism was succesfully added with address:
Via Carlo Boncompagni, 57, 20139 Milano MI, Italy

Marriott International, Inc
OK

Marriott International, Inc was skipped. 8051 NW 36th St, Doral, FL 33166, USA is not in Milano

Global Planning Architecture
OK


KeyboardInterrupt: Interrupted by user